In [8]:
from langchain_openai import ChatOpenAI
from environments.game_of_24.common.evaluate_parser import GameOf24EvaluateOutputParser
from environments.game_of_24.reflexion.evaluate_prompts import evaluate_prompt
from environments.game_of_24.reflexion.generate_prompts import openai_tools_generate_prompt

from environments.game_of_24.reflexion.self_reflection_prompts import (
    self_reflection_prompt,
)

from planning_library.strategies.reflexion import ReflexionStrategy
from environments.game_of_24.common.environment import GameOf24Env

import os

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Reflexion (new) + Game of 24"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

In [10]:
env = GameOf24Env()

In [11]:
# Reflexion hyperparameters
value_threshold = 1.0  # threshold for evaluation; when reached, the loop will exit
max_num_iterations = 20  # maximum number of iterations; when reached, the loop will exit

# other hyperparameters
model_name = "gpt-3.5-turbo"
temperature = 0.8

In [12]:
env = GameOf24Env()

In [13]:
from planning_library.action_executors import GymnasiumActionExecutor
from typing import Dict, Any


def reset_environment(inputs: Dict[str, Any]) -> None:
    env.reset(options={"numbers": [float(n) for n in inputs["inputs"].split()]})


action_executor = GymnasiumActionExecutor(env)

reflexion = ReflexionStrategy.create(
    tools=env.tools,
    action_executor=action_executor,
    value_threshold=value_threshold,
    actor_llm=ChatOpenAI(model_name=model_name, temperature=temperature),
    actor_prompt=openai_tools_generate_prompt,
    actor_agent_type="openai_tools",
    evaluator_llm=ChatOpenAI(model_name=model_name, temperature=temperature),
    evaluator_prompt=evaluate_prompt,
    evaluator_output_parser=GameOf24EvaluateOutputParser(),
    self_reflection_llm=ChatOpenAI(model_name=model_name, temperature=temperature),
    self_reflection_prompt=self_reflection_prompt,
    reset_environment=reset_environment,
    max_iterations=max_num_iterations,
)

In [14]:
env.reset(options={"numbers": [1, 1, 4, 6]})
reflexion.invoke(
    {"inputs": {"inputs": "1 1 4 6"}},
    {"recursion_limit": 1000},
)

ValueError: Unexpected message type: tool='multiply' tool_input={'number1': 6, 'number2': 4} log="\nInvoking: `multiply` with `{'number1': 6, 'number2': 4}`\n\n\n" message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_yumhWbagioeDVWOLNuHl5Y4F', 'function': {'arguments': '{"number1": 6, "number2": 4}', 'name': 'multiply'}, 'type': 'function'}, {'index': 1, 'id': 'call_06vRO1cpDSoVUw2WCWXzTefh', 'function': {'arguments': '{"number1": 1, "number2": 1}', 'name': 'multiply'}, 'type': 'function'}]})] tool_call_id='call_yumhWbagioeDVWOLNuHl5Y4F'. Use one of 'human', 'user', 'ai', 'assistant', or 'system'.